In [1]:
# DATABASE='osmium'
# HOST='127.0.0.1'
# USERNAME='kenjiwang'
# PASSWORD='5517007'
# PORT='5432'

In [2]:
from angle_emb import AnglE, Prompts
from numpy.linalg import norm
import numpy as np
import psycopg2

/Users/kenjiwang/anaconda3/envs/osmium/lib/python3.12/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [3]:
def cosine_similarity(a,b):
	return np.sum(a*b, axis=1)/(norm(a, axis=1)*norm(b, axis=1))

In [4]:
# conn = psycopg2.connect(database=DATABASE,
# 	host=HOST,
# 	user=USERNAME,
# 	password=PASSWORD,
# 	port=PORT
# )

# cursor = conn.cursor()

# cursor.execute("CREATE TABLE IF NOT EXISTS notes (id bigserial PRIMARY KEY NOT NULL, title text NOT NULL, category text NOT NULL, notes text NOT NULL)")

# conn.commit()

In [5]:
angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()
angle.set_prompt(prompt=Prompts.C)

INFO:AnglE:Prompt is set, the prompt will be automatically applied during the encoding phase. To disable prompt setting, please configure set_prompt(prompt=None)


In [6]:
def categorize_note(note, categories):
	embedded = angle.encode({"text": note}, to_numpy=True)
	categories_embedded = [angle.encode({"text": category}, to_numpy=True) for category in categories]
	scores = [cosine_similarity(category, embedded)[0] for category in categories_embedded]

	# query = f"INSERT INTO notes(title, category, notes) VALUES ('{title}', '{categories[scores.index(max(*scores))]}', '{note.replace('\'','\'\'')}')"
	# print(query)
	# cursor.execute(query)
	# conn.commit()
	return [categories[scores.index(max(*scores))], max(*scores).item()]

In [ ]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import json

hostName = "localhost"
serverPort = 3000

class MyServer(BaseHTTPRequestHandler):
    def do_OPTIONS(self):
        self.send_response(200)
        self.send_header("Access-Control-Allow-Origin", "*")
        self.send_header("Access-Control-Allow-Credentials", "*")
        self.send_header("Access-Control-Allow-Methods", "*")
        self.send_header("Access-Control-Allow-Headers", "*")
        self.end_headers()
    def do_POST(self):
        content_len = int(self.headers.get('Content-Length'))
        post_body = json.loads(self.rfile.read(content_len))
        result = categorize_note(post_body["notes"], post_body["categories"])
        self.send_response(200)
        self.send_header( "Access-Control-Allow-Origin", "*")
        self.send_header("Content-Type", "application/json")
        self.end_headers()
        self.wfile.write(bytes(json.dumps({ "bestMatch": result[0], "score": result[1] }), "utf8"))
if __name__ == "__main__":        
    webServer = HTTPServer((hostName, serverPort), MyServer)
    print("Server started http://%s:%s" % (hostName, serverPort))

    try:
        webServer.serve_forever()
    except KeyboardInterrupt:
        pass

    webServer.server_close()
    print("Server stopped.")

Server started http://localhost:3000


127.0.0.1 - - [29/Dec/2024 14:57:46] "OPTIONS / HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2024 14:57:49] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2024 14:57:49] "OPTIONS / HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2024 14:57:49] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2024 14:57:56] "OPTIONS / HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2024 14:57:56] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2024 14:57:56] "OPTIONS / HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2024 14:57:57] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2024 14:57:57] "OPTIONS / HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2024 14:57:57] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2024 14:57:57] "OPTIONS / HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2024 14:57:57] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2024 14:57:57] "OPTIONS / HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2024 14:57:57] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2024 14:57:57] "OPTIONS / HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2024 14:57:57] "POST / HTTP/1.1" 200 -
